In [1]:
#import matplotlib.pyplot as plt
#import seaborn as sns


import pandas as pd
import numpy as np
import random as rnd

from sklearn.cross_validation import KFold, cross_val_score

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



train_=pd.read_csv('../train_allcols.csv')
validate_=pd.read_csv('../validate_allcols.csv')
#test=pd.read_csv('../testwDSM.csv')

characters = pd.read_csv('chars.csv', sep=',')

train_.shape, validate_.shape, #test.shape

/Users/arc/Codes/DM-Lab/dmlab/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


((301619, 62), (148560, 62))

In [2]:
characters.shape


(9, 63)

In [3]:
train = train_.query('DSMCRIT < 14').sample(20000)
validate = validate_.query('DSMCRIT < 14').sample(10000)
#print train['DSMCRIT'].value_counts()
print train.shape

(20000, 62)


In [4]:
#frist classifier: merge 4&9 -> 4, 7&10  -> 7
train.ix[train.DSMCRIT.isin([9]), 'DSMCRIT'] = 4
train.ix[train.DSMCRIT.isin([10]), 'DSMCRIT'] = 7
validate.ix[validate.DSMCRIT.isin([9]), 'DSMCRIT'] = 4
validate.ix[validate.DSMCRIT.isin([10]), 'DSMCRIT'] = 7

/Users/arc/Codes/DM-Lab/dmlab/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/Users/arc/Codes/DM-Lab/dmlab/lib/python2.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


In [5]:
#train = train.query('SUB1 <= 10').query('SUB2 <= 10')
#validate = validate.query('SUB1 <= 10').query('SUB2 <= 10')

drop_list = ['DSMCRIT',  #'NUMSUBS'
             ]
drop_list_select = ['RACE', 'PREG', 'ARRESTS', 'PSYPROB', 'DETNLF', 'ETHNIC', 'MARSTAT', 'GENDER', 'EDUC'
                   ,'LIVARAG', 'EMPLOY', 'SUB3']

retain_list = ['RACE','PCPFLG','PRIMINC','LIVARAG','BENZFLG','HLTHINS','GENDER','ROUTE3','PRIMPAY',
               'MARSTAT','PSYPROB','ROUTE2','EMPLOY','SUB2','FRSTUSE3','FREQ3','FRSTUSE2','OTHERFLG',
               'EDUC','FREQ2','FREQ1','YEAR',
               'PSOURCE','DETCRIM','DIVISION','REGION','NOPRIOR','NUMSUBS','ALCDRUG',
               'METHUSE','FRSTUSE1','AGE','COKEFLG','OPSYNFLG','IDU','SERVSETA','ROUTE1','MARFLG',
               'MTHAMFLG','HERFLG',
               'ALCFLG','SUB1']
X_train = train[retain_list]

X_c = characters[retain_list]
Y_c = characters["DSMCRIT"]
#X_train = train.drop(drop_list + drop_list_select, axis=1)
Y_train = train["DSMCRIT"]
#X_validate = validate.drop(drop_list + drop_list_select, axis=1)
#Y_validate = validate["DSMCRIT"]
#X_test  = test.drop(drop_list, axis=1)
X_train.shape, #X_validate.shape, #X_test.shape



((20000, 42),)

In [6]:
#print X_train.columns.tolist()

In [7]:
#one hot
from sklearn import preprocessing

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(X_train)

# 3. Transform
onehotlabels = enc.transform(X_train).toarray()
X_train = onehotlabels

#onehotlabels = enc.transform(X_validate).toarray()
#X_validate = onehotlabels


X_char = enc.transform(X_c).toarray()

print X_train.shape, X_char.shape#X_validate.shape

(20000, 241) (9, 241)


In [8]:
#kfold
kf = 3

In [9]:
# Logistic Regression
logreg = LogisticRegression(n_jobs=-1)
logreg.fit(X_train, Y_train)
#Y_pred = logreg.predict(X_test)

l_acc_log = cross_val_score(logreg, X_train, Y_train, cv=kf)
acc_log = round(np.mean(l_acc_log), 3)

l_acc_log = ['%.3f' % elem for elem in l_acc_log]
print l_acc_log
print acc_log

/Users/arc/Codes/DM-Lab/dmlab/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


['0.803', '0.809', '0.805']
0.806


In [10]:
# Random Forest (slow)

random_forest = RandomForestClassifier(n_estimators=200, max_depth=20, n_jobs=-1)
random_forest.fit(X_train, Y_train)
#Y_pred = random_forest.predict(X_test)

l_acc_random_forest = cross_val_score(random_forest, X_train, Y_train, cv=kf)
acc_random_forest = round(np.mean(l_acc_random_forest), 3)

l_acc_random_forest = ['%.3f' % elem for elem in l_acc_random_forest]
print l_acc_random_forest
print acc_random_forest

['0.807', '0.818', '0.813']
0.813


In [15]:
yp_char = logreg.predict_proba(X_char)
print acc_log
print yp_char

0.806
[[  1.66428356e-04   2.00980077e-04   6.70190515e-05   8.96110366e-05
    9.27053601e-01   8.35648305e-03   6.50180203e-03   2.76022276e-02
    2.85986193e-02   1.24694532e-03   9.35095086e-05   2.27739578e-05]
 [  1.24895226e-05   2.67136207e-05   1.95053380e-02   3.43203023e-07
    9.92976350e-03   3.52815302e-03   1.39429789e-02   8.72229153e-01
    4.92319479e-02   2.51194905e-02   3.11095101e-03   3.36267771e-03]
 [  2.79895835e-05   2.94461601e-05   8.32488958e-03   1.80989571e-06
    4.92423586e-02   2.51748171e-02   5.21483553e-04   4.83641879e-03
    8.12737134e-01   2.53072689e-02   7.37941581e-02   2.22613991e-06]
 [  1.78927926e-04   1.55435328e-05   7.79004423e-02   1.69402703e-06
    4.40500111e-02   1.20777770e-02   6.90243295e-02   7.75919740e-01
    1.50207691e-02   3.10293263e-03   1.35272239e-04   2.57256052e-03]
 [  4.40493595e-06   1.02415318e-04   5.26855705e-04   7.82606139e-05
    9.19614742e-01   2.71781851e-03   8.83139090e-03   4.96336801e-03
    4.2442

In [12]:
yp_char = random_forest.predict_proba(X_char)
print acc_random_forest
print yp_char

[[  4.97933267e-04   4.92099947e-03   7.53247559e-03   8.91932275e-03
    9.00620618e-01   6.50748792e-04   6.31176789e-03   4.92441331e-02
    1.48743081e-02   1.13788712e-03   0.00000000e+00   5.28980649e-03]
 [  0.00000000e+00   0.00000000e+00   4.27168383e-04   0.00000000e+00
    1.28414425e-01   6.72612375e-03   1.10979879e-02   7.30394131e-01
    5.67492332e-02   6.03743601e-02   5.74900291e-03   6.75675676e-05]
 [  5.12195122e-03   0.00000000e+00   7.74378427e-02   1.00000000e-02
    1.53347986e-01   2.99153877e-01   2.50000000e-02   1.04166667e-02
    3.11746262e-01   6.55216802e-02   3.22537342e-02   1.00000000e-02]
 [  5.74097292e-03   3.77271153e-04   1.72544945e-02   2.66666667e-05
    1.89069389e-01   8.89345825e-04   7.31551543e-03   7.18416885e-01
    5.44120377e-02   5.78990069e-03   3.16455696e-05   6.75875479e-04]
 [  3.08300395e-04   1.07694555e-03   3.57097160e-02   1.65646927e-02
    6.45563390e-01   8.70249694e-04   1.98345178e-02   2.63125595e-02
    2.11526638e-